# Run experiments with py_experimenter

This involves specifying different active learning pipelines and evaluating them on different active learning problems. The results are then stored in a database. To specify the exact experiments and parameters to be run, the file config/exp_config.yml is used as well as parameters are filled in the function experimenter.fill_table_from_combination() below. In our case, the results will be filled into ALPBenchmark.db

For further information we refer to the docs https://tornede.github.io/py_experimenter/

In [19]:
import numpy as np
from py_experimenter.exceptions import DatabaseConnectionError
from py_experimenter.experimenter import PyExperimenter, ResultProcessor
import types
from ALP.benchmark.BenchmarkConnector import DataFileBenchmarkConnector
from ALP.benchmark.BenchmarkSuite import TabZillaBenchmarkSuite
from ALP.evaluation.experimenter.DefaultSetup import ensure_default_setup
from ALP.evaluation.experimenter.LogTableObserver import LogTableObserver, SparseLogTableObserver
from ALP.pipeline.ActiveLearningPipeline import ActiveLearningPipeline
from ALP.pipeline.Oracle import Oracle
import sqlite3
import pandas as pd

get ids of the **tabzilla benchmark suite**

In [10]:
tabzilla = TabZillaBenchmarkSuite()
tabzilla_ids = tabzilla.get_openml_dataset_ids()

setup experiment runner, that loads parameters from the grid and runs each active learning pipeline on every active learning problem

In [11]:
class ExperimentRunner:

    def __init__(self):
        pass

    def run_experiment(self, parameters: dict, result_processor: ResultProcessor, custom_config: dict):

        dbbc = DataFileBenchmarkConnector()

        connector: DataFileBenchmarkConnector = dbbc

        OPENML_ID = int(parameters["openml_id"])
        SETTING_NAME = parameters["setting_name"]
        TEST_SPLIT_SEED = int(parameters["test_split_seed"])
        TRAIN_SPLIT_SEED = int(parameters["train_split_seed"])
        SEED = int(parameters["seed"])

        setting = connector.load_setting_by_name(SETTING_NAME)
        scenario = connector.load_or_create_scenario(
            openml_id=OPENML_ID,
            test_split_seed=TEST_SPLIT_SEED,
            train_split_seed=TRAIN_SPLIT_SEED,
            seed=SEED,
            setting_id=setting.get_setting_id(),
        )

        X_l, y_l, X_u, y_u, X_test, y_test = scenario.get_data_split()

        QUERY_STRATEGY = connector.load_query_strategy_by_name(parameters["query_strategy_name"])
        LEARNER = connector.load_learner_by_name(parameters["learner_name"])

        OBSERVER = [SparseLogTableObserver(result_processor, X_test, y_test)]

        ALP = ActiveLearningPipeline(
            learner=LEARNER,
            query_strategy=QUERY_STRATEGY,
            observer_list=OBSERVER,
            num_iterations=setting.get_number_of_iterations(),
            num_queries_per_iteration=setting.get_number_of_queries(),
        )

        oracle = Oracle(X_u, y_u)
        ALP.active_fit(X_l, y_l, X_u, oracle)

run default setup, choose learning algorithms, query strategies and parameters **(chosen algorithms and their parameters are saved in alpbench/ in .json files**

to not make these experiments run for too long, we restrict ourselves to the first **4 dataset ids, 1 setting, 1 seed, 2 learning algorithms and 2 query strategies**

In [12]:
def run(run_setup=False, reset_experiments=False):

    exp_config_file = "config/exp_config.yml"

    experimenter = PyExperimenter(experiment_configuration_file_path=exp_config_file)

    if run_setup:
        benchmark_connector = DataFileBenchmarkConnector()
        ensure_default_setup(dbbc=benchmark_connector)

        benchmark_connector.cleanup()

        setting_combinations = []
        setting_combinations += [{"setting_name": "small"}]

        if reset_experiments:
            experimenter.reset_experiments("running", "failed")

        else:
            experimenter.fill_table_from_combination(
                parameters={
                    "learner_name": ["rf_entropy", "svm_rbf"],  # "xgb", "knn_3", "mlp",
                    # "tabpfn", "tabnet", "catboost"],
                    "query_strategy_name": [  # "core_set", "falcun",
                        "margin",
                        # "least_confident", "entropy",
                        # "power_margin", "bald",
                        # "power_bald",
                        # "max_entropy",
                        # "qbc_variance_ratio", "kmeans",
                        # "cluster_margin", "typ_cluster",
                        # "weighted_cluster",
                        "random",
                    ],
                    "test_split_seed": np.arange(1),
                    "train_split_seed": np.arange(1),
                    "seed": np.arange(1),
                    "openml_id": tabzilla_ids[:1],
                },
                fixed_parameter_combinations=setting_combinations,
            )

    else:
        er = ExperimentRunner()
        experimenter.execute(er.run_experiment, -1)

In [13]:
run(run_setup=True, reset_experiments=False)

2024-06-12 16:56:09,615  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-06-12 16:56:09,617  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-06-12 16:56:09,618  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-06-12 16:56:09,619  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-06-12 16:56:09,621  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-06-12 16:56:09,622  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-06-12 16:56:09,624  | py-experimenter - WARNING  | No values given for keyfield seed
2024-06-12 16:56:09,626  | py-experimenter - INFO     | Found 7 keyfields
2024-06-12 16:56:09,629  | py-experimenter - WARNING  | No resultfields given
2024-06-12 16:56:09,632  | py-experimenter - INFO     | Found 2 logtables
2024-06-12 16:56:09,634  | py-experimenter - INFO     | Found logtable results__accur

display (empty) tables

In [14]:
# Specify the path to your .db file
db_path = "ALPBenchmark.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the contents of each table
for table_name in tables:
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    display(df)
    print("\n")

# Close the connection
conn.close()

Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,margin,0,0,0,2024-06-12 16:56:09,created,None,None,None,None,None
1,2,small,11,svm_rbf,margin,0,0,0,2024-06-12 16:56:09,created,None,None,None,None,None
2,3,small,11,rf_entropy,random,0,0,0,2024-06-12 16:56:09,created,None,None,None,None,None
3,4,small,11,svm_rbf,random,0,0,0,2024-06-12 16:56:09,created,None,None,None,None,None




Contents of table sqlite_sequence:


,name,seq
0,results,4




Contents of table results__accuracy_log:


,ID,experiment_id,timestamp,model_dict




Contents of table results__labeling_log:


,ID,experiment_id,timestamp,data_dict




Contents of table results_codecarbon:


,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode


In [15]:
run(run_setup=False, reset_experiments=False)

2024-06-12 16:56:10,017  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-06-12 16:56:10,020  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-06-12 16:56:10,023  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-06-12 16:56:10,025  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-06-12 16:56:10,028  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-06-12 16:56:10,029  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-06-12 16:56:10,032  | py-experimenter - WARNING  | No values given for keyfield seed
2024-06-12 16:56:10,034  | py-experimenter - INFO     | Found 7 keyfields
2024-06-12 16:56:10,037  | py-experimenter - WARNING  | No resultfields given
2024-06-12 16:56:10,041  | py-experimenter - INFO     | Found 2 logtables
2024-06-12 16:56:10,044  | py-experimenter - INFO     | Found logtable results__accur

[codecarbon INFO @ 16:56:26] Energy consumed for all GPUs : 0.000015 kWh. Total GPU Power : 85.85569184947832 W
[codecarbon INFO @ 16:56:26] Energy consumed for all CPUs : 0.000007 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 16:56:26] 0.000024 kWh of electricity used since the beginning.
[codecarbon INFO @ 16:56:26] [setup] RAM Tracking...
[codecarbon INFO @ 16:56:26] [setup] GPU Tracking...
[codecarbon INFO @ 16:56:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:56:26] [setup] CPU Tracking...
[codecarbon WARNING @ 16:56:26] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 16:56:28] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700H but we don't know it. Please contact us.
[codecarbon INFO @ 16:56:28] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-12700H
[codecarbon INFO @ 16:56:28] >>> Tracker's metadata:
[codecarbon INFO @ 16:56:28]   Platform system: Linux-6.2.0-34-generic-x86_64-with-glibc2.35
[codeca

/home/valentin/mambaforge/envs/alp_saltbench/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/valentin/mambaforge/envs/alp_saltbench/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[codecarbon INFO @ 16:56:41] Energy consumed for RAM : 0.000002 kWh. RAM Power : 11.63014554977417 W
[codecarbon INFO @ 16:56:41] Energy consumed for all GPUs : 0.000007 kWh. Total GPU Power : 40.02500841165408 W
[codecarbon INFO @ 16:56:41] Energy consumed for all CPUs : 0.000007 kWh. Total CPU P

display (filled) table

In [17]:
# Specify the path to your .db file
db_path = "ALPBenchmark.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the contents of each table
for table_name in tables:
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    display(df)
    print("\n")

# Close the connection
conn.close()

Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,margin,0,0,0,2024-06-12 16:56:09,done,2024-06-12 16:56:10,PyExperimenter,valentin-XPS-15-9520,2024-06-12 16:56:20,None
1,2,small,11,svm_rbf,margin,0,0,0,2024-06-12 16:56:09,done,2024-06-12 16:56:20,PyExperimenter,valentin-XPS-15-9520,2024-06-12 16:56:26,None
2,3,small,11,rf_entropy,random,0,0,0,2024-06-12 16:56:09,done,2024-06-12 16:56:26,PyExperimenter,valentin-XPS-15-9520,2024-06-12 16:56:35,None
3,4,small,11,svm_rbf,random,0,0,0,2024-06-12 16:56:09,done,2024-06-12 16:56:35,PyExperimenter,valentin-XPS-15-9520,2024-06-12 16:56:41,None




Contents of table sqlite_sequence:


,name,seq
0,results,4
1,results__labeling_log,4
2,results__accuracy_log,4
3,results_codecarbon,4




Contents of table results__accuracy_log:


,ID,experiment_id,timestamp,model_dict
0,1,1,2024-06-12 16:56:20,"{""0"": {""iteration"": 0, ""test_f1"": 0.71268675, ..."
1,2,2,2024-06-12 16:56:26,"{""0"": {""iteration"": 0, ""test_f1"": 0.78834966, ..."
2,3,3,2024-06-12 16:56:35,"{""0"": {""iteration"": 0, ""test_f1"": 0.70086908, ..."
3,4,4,2024-06-12 16:56:41,"{""0"": {""iteration"": 0, ""test_f1"": 0.78834966, ..."




Contents of table results__labeling_log:


,ID,experiment_id,timestamp,data_dict
0,1,1,2024-06-12 16:56:20,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
1,2,2,2024-06-12 16:56:26,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
2,3,3,2024-06-12 16:56:35,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
3,4,4,2024-06-12 16:56:40,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."




Contents of table results_codecarbon:


,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode
0,1,1,2024-06-12T16:56:20,codecarbon,8253e3ea-b5a6-4564-a256-38ec99167de7,4.153006,0.000027,0.000007,42.5,10.104948,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013721,machine,N,1.0,0
1,2,2,2024-06-12T16:56:26,codecarbon,1aa21a13-9771-4eda-93af-417088e40fdf,0.641145,0.000009,0.000014,42.5,85.855692,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013721,machine,N,1.0,0
2,3,3,2024-06-12T16:56:35,codecarbon,5380d105-bc5e-42e5-a6bd-ccfc09a7021a,3.857886,0.000024,0.000006,42.5,7.256093,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013721,machine,N,1.0,0
3,4,4,2024-06-12T16:56:41,codecarbon,bf002ac0-42ad-4fb8-b9fd-394fb2c99337,0.603670,0.000006,0.000009,42.5,40.025008,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5683,48.1663,31.013721,machine,N,1.0,0
